In [1]:
import os
import pandas as pd
import numpy as np
from utils.mapping import mapDataframeToGridClosest
from utils.global_co2 import getGlobalCo2ForYear

In [14]:
folder_path = "../../../../../../../Volumes/T7 Shield/exp_3"
output_path = "../../../../../../../Volumes/T7 Shield/preprocessed_exp3"
# folder_path = "../../data"
# output_path = "../../data/preprocessed_new"

# experiment 1
# file_prefix = "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.LP04-KLP002.hind_"

# experiment 3
file_prefix = "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.LP11-KLP009.clim_"

# experiment 5
# file_prefix = "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.LP04-KLP002.wind_"

# experiment 6
# file_prefix = "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.L.LP03-KLP004_"


features = ['SST', 'SAL', 'ice_frac', 'mixed_layer_depth', 'heat_flux_down', 'water_flux_up', 'stress_X', 'stress_Y', 'currents_X', 'currents_Y',
            'month','tmask','year','nav_lat', 'nav_lon','global_co2']
targets = ['fco2','fco2_pre','co2flux','co2flux_pre']

def regrid_and_store(year, index):
    input_file = os.path.join(folder_path, f"{file_prefix}{year}_df.pkl")
    df = pd.read_pickle(input_file)

    df["year"] = year
    df["month"] = df["time_counter"].apply(lambda x: x.month)
    df['global_co2'] = getGlobalCo2ForYear(year)
    df = df[features+targets]

    if df.isna().any().any():
        print("contains NaN values")
        print(df.isna().sum())


    for month in range(1, 13):
        month_data = df.loc[(df['month'] == month)]

        features_array = mapDataframeToGridClosest(month_data, features)
        target_array = mapDataframeToGridClosest(month_data, targets)

        np.save(os.path.join(output_path, f"{index+month}_features.npy"), features_array)
        np.save(os.path.join(output_path, f"{index+month}_targets.npy"), target_array)

In [15]:
i = 0

for year in range(1958, 2019):
    print(year)
    regrid_and_store(year, i)
    i = i+12

1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
